In [230]:
import requests
import pandas as pd
import time
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('lang=ko_KR')

base_url='https://movie.naver.com'
current='https://movie.naver.com/movie/running/current.nhn'
driver = webdriver.Chrome('./chromedriver', options=chrome_options)
driver.implicitly_wait(3)

driver.get(current)

page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

url_list=soup.select('ul.lst_detail_t1 dt a')

#영화 목록
df_movie=pd.DataFrame(columns=['name','url'])
for i in range(len(url_list)):
    temp=pd.Series([url_list[i].text, base_url+url_list[i].get('href')], index=df_movie.columns)
    df_movie=df_movie.append(temp,ignore_index=True)

#영화 별 리뷰
df_movie_review=pd.DataFrame(columns=['mid','user','rate','review'])
for key in tqdm(range(len(df_movie))):
    try:
        iframe_page='https://movie.naver.com/movie/bi/mi/point.nhn?code='+str(df_movie.iloc[key,1][df_movie.iloc[key,1].find('=')+1:])
        driver.get(iframe_page)
        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')

        review_page=base_url+soup.select('iframe.ifr')[0].get('src')
        driver.get(review_page)
        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')

        paging=soup.select('div.paging a')
        page_list=[base_url+paging[i].get('href') for i in range(len(paging)-1)]

        #리뷰에 paging 있음, paging 별 리뷰 10개씩 있음
        for _page in page_list:
            driver.get(_page)
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')

            temp_df=pd.DataFrame(columns=['mid','user','rate','review'])
            for i in range(len(soup.select('div.star_score em'))):
                #user, rate 정보
                rate=soup.select('div.star_score em')[i].text
                user=soup.select('div.score_reple dl span')[i].text

                temp_series=pd.Series([key,user,rate,None], index=df_movie_review.columns)
                temp_df=temp_df.append(temp_series, ignore_index=True)

            #review는 복잡해서 따로 가져옴
            span_list=soup.select('div.score_reple p span')
            review_list=[]
            for span in span_list:
                try:
                    if '_filtered' in span.get('id'):
                        review_list.append(span)
                except:
                    pass

            reviews=[]
            for review in review_list:
                try:
                    #긴 리뷰
                    reviews.append(review.a.get('data-src'))
                except:
                    #짧은 리뷰
                    reviews.append(review.text.replace('\n','').replace('\t',''))

            temp_df['review']=reviews
            df_movie_review=df_movie_review.append(temp_df)
    except:
        print(key)
        
df_movie.to_csv('current_movie.csv')
df_movie_review.to_csv('current_movie_review.csv')

61
130
141



In [231]:
df_movie_review

,mid,user,rate,review
0,0,허현(heoh****),6,평점 알바 특. 내용얘기보다 배우 연기 좋단말만 주구장창함...
1,0,황금마차(goga****),1,평점 알바풀었나...
2,0,케구리(jhw6****),2,배우들의 연기만 좋았음 하지만 단점이 확실히 보인다. 영화 초반부 사건이 터지고 주...
3,0,깁스(volk****),1,실망 뻔한스토리에 억지스럼
4,0,달고나(mjh5****),1,뻔한 신파극 평점알바풀었나 배우들 연기만 좋음
...,...,...,...,...
5,153,안경잡이(ayan****),8,초등학교때 이 영화 보고 1주일을 앓아 누웠던 기억이..ㅎㄷㄷ 그리고 시간이 많이 ...
6,153,개럭(jtot****),9,처음에는 얼굴 가리면서 봤지만 어느새 적응하고 있는 내 모습.
7,153,뉴존(newz****),10,지금 생각해도 당시 획기적인 아이디어에 과학의 신선함과 공포를 한번에 느끼게해준 ...
8,153,무탈 Watanabe(yaso****),8,감독 특유의 기괴한 상상. 그래도 요즘 웬만한 스릴러 영화보다 훨씬 낫다.


In [233]:
df_movie.iloc[[61,130,141],:]

,name,url
61,퍼니 게임,https://movie.naver.com/movie/bi/mi/basic.nhn?...
130,나는 네가 지난 여름에 한 일을 알고 있다,https://movie.naver.com/movie/bi/mi/basic.nhn?...
141,스크림,https://movie.naver.com/movie/bi/mi/basic.nhn?...
